In [5]:
from google.colab import drive
drive.mount('/content/drive')





Mounted at /content/drive


In [6]:
!pip install pandas numpy transformers scikit-learn datasets torch  pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [7]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset, DatasetDict
import torch


In [5]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/ghiwane nlp/Dataset-SA.csv')  # Replace with your actual filename

In [6]:
df.head()

,product_name,product_price,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,neutral


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205052 entries, 0 to 205051
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   product_name   205052 non-null  object
 1   product_price  205052 non-null  object
 2   Rate           205052 non-null  object
 3   Review         180388 non-null  object
 4   Summary        205041 non-null  object
 5   Sentiment      205052 non-null  object
dtypes: object(6)
memory usage: 9.4+ MB


In [8]:
df.describe()

,product_name,product_price,Rate,Review,Summary,Sentiment
count,205052,205052,205052,180388,205041,205052
unique,958,525,8,1324,92923,3
top,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,wonderful,good,positive
freq,6005,9150,118765,9016,17430,166581


In [9]:
# Convert 'Rate' to numeric
df['Rate'] = pd.to_numeric(df['Rate'], errors='coerce')
df.dropna(subset=['Review', 'Rate'], inplace=True)


In [10]:
#Create 'Sentiment' column
def map_sentiment(rating):
    if rating >= 4:
        return "positive"
    elif rating <= 2:
        return "negative"
    else:
        return "neutral"

df['Sentiment'] = df['Rate'].apply(map_sentiment)


In [11]:
# Select and rename columns
df = df[['Review', 'Sentiment']]
df.columns = ['text', 'label']

# Print label distribution
print(df['label'].value_counts())

label
positive    142616
negative     23745
neutral      14024
Name: count, dtype: int64


In [12]:
# Load the tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Tokenize the text
tokenized_text = tokenizer(
    df['text'].tolist(),
    padding="max_length",
    truncation=True,
    max_length=128,  # Adjust if necessary
    return_tensors="pt"
)

In [ ]:
# Convert tokenized tensors to lists and add to DataFrame.
tokenized_data = df.copy()
tokenized_data['input_ids'] = [t.tolist() for t in tokenized_text['input_ids']]
tokenized_data['attention_mask'] = [t.tolist() for t in tokenized_text['attention_mask']]


In [ ]:
# Convert labels to numerical format (Important: AFTER tokenization)
label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
tokenized_data['label'] = tokenized_data['label'].map(label_mapping)


In [ ]:
#Convert to Hugging Face Dataset
tokenized_datasets = Dataset.from_pandas(tokenized_data)

In [ ]:
tokenized_datasets

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask', '__index_level_0__'],
    num_rows: 180385
})

In [ ]:
# Split the data (using the original pandas DataFrame)
train_valid_df, test_df = train_test_split(tokenized_data, test_size=0.2, random_state=42) #No more stratification here as this is a pandas DataFrame.
train_df, valid_df = train_test_split(train_valid_df, test_size=0.1, random_state=42) #No more stratification here either.



In [ ]:
#Convert splits to Datasets. Create DatasetDict.
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

data_dict = DatasetDict({'train':train_dataset, 'validation': valid_dataset, 'test': test_dataset}) #Combine Datasets into a DatasetDict.

In [ ]:
# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/ghiwane nlp/tokenizer")


('/content/drive/MyDrive/ghiwane nlp/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/ghiwane nlp/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/ghiwane nlp/tokenizer/vocab.txt',
 '/content/drive/MyDrive/ghiwane nlp/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/ghiwane nlp/tokenizer/tokenizer.json')

In [ ]:
from datasets import Dataset

# Save each split to disk
train_dataset.save_to_disk("/content/drive/MyDrive/ghiwane nlp/train_dataset")
valid_dataset.save_to_disk("/content/drive/MyDrive/ghiwane nlp/valid_dataset")
test_dataset.save_to_disk("/content/drive/MyDrive/ghiwane nlp/test_dataset")


Saving the dataset (0/1 shards):   0%|          | 0/129877 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14431 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/36077 [00:00<?, ? examples/s]

In [ ]:
# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,              # Adjust as needed
    per_device_train_batch_size=16,  # Adjust as needed
    per_device_eval_batch_size=64,   # Adjust as needed
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",  # Corrected argument name
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",      # Use weighted f1 for imbalanced data
    report_to="none"  # Disable WandB integration
    # push_to_hub=True,  # Uncomment if using Hugging Face Hub. Requires login.
    # push_to_hub_model_id="your-model-id" # Use your model id if push_to_hub=True. Requires login.
)


In [ ]:
# Define the evaluation metric calculation function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')  # Use weighted F1
    return {"accuracy": accuracy, "f1": f1}


In [ ]:
# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_dict['train'],  # Access train set from DatasetDict
    eval_dataset=data_dict['validation'],  # Access validation set from DatasetDict
    compute_metrics=compute_metrics  # Pass the metrics function
)

In [ ]:
# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.026400,0.051857,0.986626,0.986488


TrainOutput(global_step=8118, training_loss=0.06984816343381535, metrics={'train_runtime': 2963.3081, 'train_samples_per_second': 43.828, 'train_steps_per_second': 2.74, 'total_flos': 8543095338564864.0, 'train_loss': 0.06984816343381535, 'epoch': 1.0})

In [ ]:
# Evaluate the model on the test set
predictions = trainer.predict(data_dict['test']) #Make sure to pass a Dataset object to predict.
pred_labels = np.argmax(predictions.predictions, axis=-1)
true_labels = predictions.label_ids
test_f1 = f1_score(true_labels, pred_labels, average='weighted')
test_accuracy = accuracy_score(true_labels, pred_labels)
print(f"Test F1: {test_f1}")
print(f"Test Accuracy: {test_accuracy}")


Test F1: 0.9843988752080076
Test Accuracy: 0.9845885190010256


In [ ]:
# Save the fine-tuned model
trainer.save_model("/kaggle/working/best_flipkart_model")

In [ ]:
# Save the fine-tuned model
trainer.save_model("/content/drive/MyDrive/ghiwane nlp/best_flipkart_model")

In [18]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/ghiwane nlp/tokenizer")


from datasets import load_from_disk

# Load each dataset split
train_dataset = load_from_disk("/content/drive/MyDrive/ghiwane nlp/train_dataset")
valid_dataset = load_from_disk("/content/drive/MyDrive/ghiwane nlp/valid_dataset")
test_dataset = load_from_disk("/content/drive/MyDrive/ghiwane nlp/test_dataset")


In [19]:

data_dict = DatasetDict({'train':train_dataset, 'validation': valid_dataset, 'test': test_dataset}) #Combine Datasets into a DatasetDict.

In [20]:
# Load the fine-tuned model and tokenizer
model_path = "/content/drive/MyDrive/ghiwane nlp/best_flipkart_model"
# tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Create a new Trainer instance with the loaded model
trainer = Trainer(model=model, tokenizer=tokenizer)


<ipython-input-20-3ae81ab8beb0>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer)


In [21]:
# Evaluate the model on the test set
predictions = trainer.predict(data_dict['test']) #Make sure to pass a Dataset object to predict.
pred_labels = np.argmax(predictions.predictions, axis=-1)
true_labels = predictions.label_ids
test_f1 = f1_score(true_labels, pred_labels, average='weighted')
test_accuracy = accuracy_score(true_labels, pred_labels)
print(f"Test F1: {test_f1}")
print(f"Test Accuracy: {test_accuracy}")


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Test F1: 0.9843988752080076
Test Accuracy: 0.9845885190010256


# Topic Modeling:



*1. Preprocess the Text*

We’ll preprocess the text data by tokenizing and removing stopwords, keeping only alphanumeric tokens.

In [17]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/ghiwane nlp/Dataset-SA.csv')  # Replace with your actual filename

In [18]:
df

,product_name,product_price,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,neutral
...,...,...,...,...,...,...
205047,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,must buy!,good product,positive
205048,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,super!,nice,positive
205049,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,3,nice,very nice and fast delivery,positive
205050,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,just wow!,awesome product,positive


In [19]:
# Convert 'Rate' to numeric
df['Rate'] = pd.to_numeric(df['Rate'], errors='coerce')
df.dropna(subset=['Summary', 'Rate'], inplace=True)


In [20]:
#Create 'Sentiment' column
def map_sentiment(rating):
    if rating >= 4:
        return "positive"
    elif rating <= 2:
        return "negative"
    else:
        return "neutral"

df['Sentiment'] = df['Rate'].apply(map_sentiment)


In [21]:
# Select and rename columns
df = df[['Summary', 'Sentiment']]
df.columns = ['text', 'label']

# Print label distribution
print(df['label'].value_counts())

label
positive    160649
negative     27790
neutral      16599
Name: count, dtype: int64


In [22]:
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from gensim import corpora, models
import pandas as pd

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Extract review texts
review_texts = df['text'].tolist()

# Preprocess text
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]  # Remove stopwords and non-alphanumeric
    return tokens

preprocessed_texts = [preprocess_text(text) for text in review_texts]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
from nltk.tokenize import word_tokenize, RegexpTokenizer

# Use a regular expression tokenizer as an alternative
tokenizer = RegexpTokenizer(r'\w+')

def preprocess_text(text):
    tokens = tokenizer.tokenize(text.lower())  # Tokenize using regex
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

preprocessed_texts = [preprocess_text(text) for text in df['text'].tolist()]


2. Create Bag-of-Words and Dictionary

We’ll create a dictionary and bag-of-words representation of the preprocessed texts.



In [26]:
# Create a dictionary and corpus
dictionary = corpora.Dictionary(preprocessed_texts)  # Map each word to a unique ID
corpus = [dictionary.doc2bow(text) for text in preprocessed_texts]  # Convert texts to bag-of-words format

print(f"Number of unique tokens: {len(dictionary)}")
print(f"Number of documents: {len(corpus)}")


Number of unique tokens: 47435
Number of documents: 205038


3. Apply LDA

Use Gensim's LDA model to identify topics in the reviews.

In [27]:
# Train the LDA model
num_topics = 3  # Adjust the number of topics based on your dataset
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10, random_state=42)

# Print the topics
print("Identified Topics:")
for idx, topic in lda_model.print_topics(num_words=10):
    print(f"Topic {idx + 1}: {topic}")


Identified Topics:
Topic 1: 0.022*"small" + 0.017*"size" + 0.016*"money" + 0.010*"like" + 0.009*"value" + 0.009*"great" + 0.009*"one" + 0.008*"go" + 0.008*"also" + 0.008*"worth"
Topic 2: 0.252*"product" + 0.168*"nice" + 0.039*"flipkart" + 0.030*"awesome" + 0.027*"super" + 0.024*"excellent" + 0.024*"bad" + 0.023*"happy" + 0.017*"delivery" + 0.016*"thank"
Topic 3: 0.429*"good" + 0.129*"quality" + 0.042*"best" + 0.041*"price" + 0.022*"ok" + 0.015*"superb" + 0.013*"one" + 0.012*"poor" + 0.009*"material" + 0.008*"looking"


4. Visualize Topics

Use
```
# pyLDAvis
```
for interactive topic visualization.


In [29]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
pyLDAvis.enable_notebook()
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary)

# Save the visualization to an HTML file
pyLDAvis.save_html(lda_vis, '/content/drive/MyDrive/ghiwane nlp/lda_topics.html')
print("LDA visualization saved as 'lda_topics.html'")


LDA visualization saved as 'lda_topics.html'


5. Summarize Topics

Create a human-readable summary of the identified topics.

In [30]:
# Summarize the topics
topic_summary = []
for idx, topic in lda_model.show_topics(num_topics=num_topics, num_words=5, formatted=False):
    words = [word for word, _ in topic]
    topic_summary.append(f"Topic {idx + 1}: {' | '.join(words)}")

# Display topic summaries
print("\nTopic Summaries:")
for summary in topic_summary:
    print(summary)



Topic Summaries:
Topic 1: small | size | money | like | value
Topic 2: product | nice | flipkart | awesome | super
Topic 3: good | quality | best | price | ok


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

# Additional domain-specific stopwords
custom_stopwords = set(stopwords.words('english')).union({"product", "good", "nice", "flipkart", "ok"})

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in custom_stopwords]
    return tokens

# Apply preprocessing
preprocessed_texts = [preprocess_text(text) for text in df['text']]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [32]:
# Create a dictionary and corpus
dictionary = corpora.Dictionary(preprocessed_texts)  # Map each word to a unique ID
corpus = [dictionary.doc2bow(text) for text in preprocessed_texts]  # Convert texts to bag-of-words format

print(f"Number of unique tokens: {len(dictionary)}")
print(f"Number of documents: {len(corpus)}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Number of unique tokens: 45544
Number of documents: 205038


In [43]:
from gensim.models import CoherenceModel

# Find the optimal number of topics
def compute_coherence(corpus, dictionary, texts, start=2, limit=10, step=1):
    coherence_values = []
    for num_topics in range(start, limit, step):
        model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10, random_state=42)
        coherence_model = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append((num_topics, coherence_model.get_coherence()))
    return coherence_values

# Compute coherence
coherence_scores = compute_coherence(corpus, dictionary, preprocessed_texts)

# Print coherence scores
for num_topics, coherence in coherence_scores:
    print(f"Num Topics: {num_topics}, Coherence Score: {coherence}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ValueError: not enough values to unpack (expected 2, got 1)

# Topic Modeling:

In [34]:
# Creating example documents
doc_1 = "A whopping 96.5 percent of water on Earth is in our oceans, covering 71 percent of the surface of our planet. And at any given time, about 0.001 percent is floating above us in the atmosphere. If all of that water fell as rain at once, the whole planet would get about 1 inch of rain."

doc_2 = "One-third of your life is spent sleeping. Sleeping 7-9 hours each night should help your body heal itself, activate the immune system, and give your heart a break. Beyond that--sleep experts are still trying to learn more about what happens once we fall asleep."

doc_3 = "A newborn baby is 78 percent water. Adults are 55-60 percent water. Water is involved in just about everything our body does."

doc_4 = "While still in high school, a student went 264.4 hours without sleep, for which he won first place in the 10th Annual Great San Diego Science Fair in 1964."

doc_5 = "We experience water in all three states: solid ice, liquid water, and gas water vapor."

# Create corpus
corpus = [doc_1, doc_2, doc_3, doc_4, doc_5]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Preprocessing

In [35]:
# Code source: https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/

import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

# remove stopwords, punctuation, and normalize the corpus
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = "".join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

clean_corpus = [clean(doc).split() for doc in corpus]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In the code above we:


1.  Imported the necessary libraries and downloaded stopwords and wordnet


2.  Defined the English stopwords


3.  Instantiated the set of punctuation we want to exclude

4.  Created the instance of the wordnet lemmatizer


1.   Created a function to remove stopwords and punctuation and to lemmatize the documents.

2.  Applied the clean function to each document in the corpus.




But this still doesn’t mean we’re ready.

Before we can use this data as input to a LDA or LSA model, it must be converted to a term-document matrix. A term-document matrix is merely a mathematical representation of a set of documents and the terms contained within them.

It’s created by counting the occurrence of every term in each document and then normalizing the counts to create a matrix of values that can be used for analysis.

To do this in Python, we’re going to leverage the Gensim library.

In [36]:
from gensim import corpora

# Creating document-term matrix
dictionary = corpora.Dictionary(clean_corpus)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in clean_corpus]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Modeling
The first model we’ll use in LSA:

In [41]:
from gensim.models import LsiModel

# LSA model
lsa = LsiModel(doc_term_matrix, num_topics=3, id2word = dictionary)

# LSA model
lsa.print_topics(num_topics=3, num_words=3)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning: `scipy.sparse.sparsetools.csc_matvecs` is deprecated along with the `scipy.sparse.sparsetools` namespace. `scipy.sparse.sparsetools.csc_matvecs` will be removed in SciPy 1.14.0, and the `scipy.sparse.sparsetools` namespace will be removed in SciPy 2.0.0.
  sparsetools.csc_matvecs(


[(0, '0.555*"water" + 0.489*"percent" + 0.239*"planet"'),
 (1, '0.361*"sleeping" + 0.215*"still" + 0.215*"hour"'),
 (2, '-0.562*"water" + 0.231*"planet" + 0.231*"rain"')]

This outputs the topics (each line) with individual topic terms (terms) and their weights.

Let’s try it with LDA:

In [42]:
from gensim.models import LdaModel

# LDA model
lda = LdaModel(doc_term_matrix, num_topics=3, id2word = dictionary)

# Results
lda.print_topics(num_topics=3, num_words=3)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, '0.030*"hour" + 0.026*"still" + 0.025*"sleeping"'),
 (1, '0.058*"water" + 0.056*"percent" + 0.033*"rain"'),
 (2, '0.076*"water" + 0.029*"percent" + 0.027*"body"')]

## What is Topic Modeling Used For?
By removing manual and repetitive tasks, topic modeling can easily and inexpensively speed up processes in a simple fashion. Here are a few examples:

# Tagging support tickets
Topic modeling can be used to help customer service staff analyze support queries to identify the primary issues and determine any that repeatedly occur. Based on that data, they may be able to create more informative self-serve content or help customers directly.

# Enhancing customer experience
Topic modeling may be used to tag conversations such that they may be routed to the most appropriate team. For example, a conversation that includes words such as “pricing,” “subscription,” “renewal,” etc. could be sent straight to the accounting department for support.

# Topic Modeling vs Other Techniques
#Topic modeling vs clustering

Topic modeling is used to discover latent topics that exist within a collection of documents. This involves identifying patterns in the words and phrases that appear in documents and grouping them into topics based on how similar they are.

Contrastingly, clustering is a technique used to group similar objects based on a measure of similarity. Such methods are employed to discover patterns and structure in data by grouping together similar data points.

Though both approaches can uncover patterns in text data, they have different goals. Topic modeling is concerned with identifying latent topics in a collection of documents, while clustering is concerned with grouping similar data points together.

#Topic modeling vs text classification

Text classification, though a natural language processing technique, falls under the category of supervised learning. Namely, text classification is employed to label predefined categories or a given piece of text. In order for the model to achieve this feat, it must first learn from a labeled data set before it can be used to make predictions on new, unseen text samples.

On the other hand, topic modeling is an unsupervised learning technique used to find the underlying topics in a collection of text documents. This means it doesn’t have to learn from a labeled dataset.

Thus, the difference between the two methods is text classification is used to assign predefined labels to text, whereas topic modeling discovers the underlying topics in a collection of documents.